In [12]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import davies_bouldin_score, silhouette_score

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

from itertools import product
from energy_consumption_architecture.utils.paths import data_dir
# from kneebow.rotor import Rotor


## Funciones auxiliares

In [43]:
def optimal_k_selection(X, max_k=10):
    """
    Calcula el número óptimo de clusters usando el índice de silueta y el método del codo.

    Parámetros:
    - X: Dataset (matriz de características).
    - max_k: Número máximo de clusters a evaluar. Por defecto es 10.

    Retorna:
    - optimal_k: Número óptimo de clusters seleccionado.
    """

    Sum_of_squared_distances = []
    silhouette_scores = []
    K_range = range(2, max_k + 1)

    for k in K_range:
        km = KMeans(n_clusters=k, random_state=42)
        y = km.fit_predict(X)
        Sum_of_squared_distances.append(km.inertia_)
        silhouette_scores.append(silhouette_score(X, y))

    # Determinación del número óptimo de clusters según el índice de silueta
    optimal_k_silhouette = K_range[np.argmax(silhouette_scores)]

    # Determinación del número óptimo de clusters según el método del codo
    inertia_differences = np.diff(Sum_of_squared_distances)
    optimal_k_elbow = K_range[np.argmin(inertia_differences) + 1]  # +1 para ajustar el índice

    # Selección de un solo valor de K
    if optimal_k_silhouette == optimal_k_elbow:
        optimal_k = optimal_k_silhouette
    else:
        optimal_k = optimal_k_silhouette  # En caso de diferencia, priorizamos el índice de silueta

    return optimal_k


In [44]:
def optimal_dbscan_params(features, eps_range=(0.05, 0.2, 0.05), min_samples_range=(3, 12)):
    """
    Encuentra los parámetros óptimos para DBSCAN (eps y min_samples) basados en el índice de silueta.

    Parámetros:
    - features: Matriz de características para clustering.
    - eps_range: Tupla con rango de valores de eps (inicio, fin, paso).
    - min_samples_range: Tupla con valores de min_samples (inicio, fin).

    Retorna:
    - Mejor combinación de (eps, min_samples) basada en el índice de silueta.
    """

    # Paso 1: Gráfica de distancia de vecinos para estimación inicial de `eps`
    neighbors = NearestNeighbors(n_neighbors=2)
    neighbors_fit = neighbors.fit(features)
    distances, _ = neighbors_fit.kneighbors(features)

    # Paso 2: Pruebas de diferentes combinaciones de eps y min_samples
    eps_values = np.arange(*eps_range)
    min_samples_values = np.arange(*min_samples_range)
    dbscan_params = list(product(eps_values, min_samples_values))
    
    best_params = (None, None)
    best_sil_score = -1  # Inicializamos con un valor muy bajo

    # Almacenamos métricas para análisis adicional
    results = {
        'Eps': [],
        'Min_samples': [],
        'Silhouette Score': [],
        'Clusters': []
    }

    for eps, min_samples in dbscan_params:
        db = DBSCAN(eps=eps, min_samples=min_samples)
        labels = db.fit_predict(features)

        # Solo evaluamos si hay más de un cluster
        if len(set(labels)) > 1:
            try:
                sil_score = silhouette_score(features, labels)
            except ValueError:
                sil_score = 0  # Si no se puede calcular, asignamos 0
        else:
            sil_score = 0

        # Guardamos resultados en el diccionario
        results['Eps'].append(eps)
        results['Min_samples'].append(min_samples)
        results['Silhouette Score'].append(sil_score)
        results['Clusters'].append(len(set(labels)))

        # Actualizar los mejores parámetros si encontramos un mejor índice de silueta
        if sil_score > best_sil_score:
            best_sil_score = sil_score
            best_params = (eps, min_samples)

    # Convertimos resultados en DataFrame para análisis
    df_results = pd.DataFrame(results)

    # Resultados de pivot para visualización opcional
    pivot_sil_score = pd.pivot_table(df_results, values='Silhouette Score', columns='Eps', index='Min_samples')
    pivot_clusters = pd.pivot_table(df_results, values='Clusters', columns='Eps', index='Min_samples')

    return best_params


In [45]:
def optimal_clusters_hierarchical(features, method='ward', last_n=10):
    """
    Calcula el número óptimo de clusters para clustering jerárquico usando la aceleración en la linkage matrix.

    Parámetros:
    - features: Matriz de características para clustering.
    - method: Método de linkage. Por defecto es 'ward'.
    - last_n: Número de fusiones a considerar para calcular el número óptimo de clusters. Por defecto es 10.

    Retorna:
    - Número óptimo de clusters.
    """

    # Calcular la linkage matrix
    mergings = linkage(features, method=method)

    # Obtener las alturas de los últimos 'last_n' clusters
    last = mergings[-last_n:, 2]
    last_rev = last[::-1]

    # Calcular la aceleración (segunda derivada)
    acceleration = np.diff(last, 2)  # Segunda derivada de las alturas
    acceleration_rev = acceleration[::-1]

    # Encontrar el número óptimo de clusters
    optimal_k = acceleration_rev.argmax() + 2  # +2 porque se pierde una posición en cada derivada

    return optimal_k


In [46]:
# Función para ejecutar K-Means y calcular métricas
def kmeans_clustering(X, max_k=10):
    optimal_k = optimal_k_selection(X, max_k=max_k)
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    clusters = kmeans.fit_predict(X)
    num_clusters = kmeans.n_clusters
    silhouette = silhouette_score(X, clusters)
    davies_bouldin = davies_bouldin_score(X, clusters)
    return clusters, silhouette, davies_bouldin, "K-Means",num_clusters

# Función para ejecutar DBSCAN y calcular métricas
def dbscan_clustering(X, eps_range=(0.05, 0.2, 0.05), min_samples_range=(3, 12)):
    eps_min_samples = optimal_dbscan_params(X, eps_range=eps_range, min_samples_range=min_samples_range)
    dbscan = DBSCAN(eps=eps_min_samples[0], min_samples=eps_min_samples[1])
    clusters = dbscan.fit_predict(X)
    num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    if len(set(clusters)) > 1:
        silhouette = silhouette_score(X, clusters)
    else:
        silhouette = -1  # Silhouette no es aplicable si hay un solo cluster
    davies_bouldin = davies_bouldin_score(X, clusters)
    return clusters, silhouette, davies_bouldin, "DBSCAN",num_clusters

# Función para ejecutar Clustering Jerárquico y calcular métricas
def hierarchical_clustering(X, last_n=10):
    optimal_k = optimal_clusters_hierarchical(X, method='ward', last_n=last_n)
    hierarchical = AgglomerativeClustering(n_clusters=optimal_k)
    clusters = hierarchical.fit_predict(X)
    num_clusters = hierarchical.n_clusters
    silhouette = silhouette_score(X, clusters)
    davies_bouldin = davies_bouldin_score(X, clusters)
    return clusters, silhouette, davies_bouldin, "Hierarchical",num_clusters


In [47]:
def standardize_data(data):
    # Seleccionar todas las columnas numéricas
    numeric_columns = data.select_dtypes(include=['number']).columns
    scaler = StandardScaler()
    # Escalar únicamente las columnas numéricas
    data_scaled = scaler.fit_transform(data[numeric_columns])
    # Devolver un DataFrame con las columnas estandarizadas
    return pd.DataFrame(data_scaled, columns=numeric_columns)

In [48]:

# Función para determinar el número óptimo de componentes principales
def determine_optimal_pca_components(df, variance_threshold=0.95):
    """
    Determina el número óptimo de componentes principales basándose en la varianza explicada acumulativa.

    Parámetros:
    - df: DataFrame estandarizado con las características numéricas.
    - variance_threshold: Umbral mínimo de varianza explicada acumulativa (por defecto: 0.95).

    Retorna:
    - optimal_components: Número óptimo de componentes principales.
    - explained_variance: Lista de la varianza explicada para cada componente.
    - cumulative_variance: Varianza explicada acumulativa.
    """
    pca = PCA()
    pca.fit(df)
    
    explained_variance = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance)
    
    # Determinar el número mínimo de componentes para alcanzar el umbral
    optimal_components = np.argmax(cumulative_variance >= variance_threshold) + 1
    
    return optimal_components, explained_variance, cumulative_variance

# Función para aplicar PCA
def apply_pca(df, n_components):
    """
    Aplica PCA para reducir la dimensionalidad de los datos.

    Parámetros:
    - df: DataFrame estandarizado con las características numéricas.
    - n_components: Número de componentes principales a conservar.

    Retorna:
    - principal_df: DataFrame con las nuevas características principales.
    - pca_model: Modelo PCA ajustado.
    """
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(df)
    principal_df = pd.DataFrame(principal_components, columns=[f'PC{i+1}' for i in range(principal_components.shape[1])])
    return principal_df, pca

# Pipeline para PCA
def pca_pipeline(df, variance_threshold=0.95):
    """
    Pipeline completo para estandarizar los datos, determinar el número óptimo de componentes
    y aplicar PCA.

    Parámetros:
    - df: DataFrame con las características numéricas originales.
    - variance_threshold: Umbral mínimo de varianza explicada acumulativa para seleccionar componentes.

    Retorna:
    - principal_df: DataFrame con las características principales resultantes.
    - optimal_components: Número óptimo de componentes principales.
    - explained_variance: Lista de la varianza explicada para cada componente.
    - cumulative_variance: Varianza explicada acumulativa.
    - pca_model: Modelo PCA ajustado.
    """
    # Estandarizar los datos
    df_scaled = standardize_data(df)
    
    # Determinar el número óptimo de componentes principales
    optimal_components, explained_variance, cumulative_variance = determine_optimal_pca_components(df_scaled, variance_threshold)
    
    # Aplicar PCA con el número óptimo de componentes
    principal_df, pca_model = apply_pca(df_scaled, n_components=optimal_components)
    
    return principal_df, optimal_components, explained_variance, cumulative_variance, pca_model

In [49]:
def evaluate_models(models_metrics):
    # Convertir las métricas a DataFrame, incluyendo el número de clusters
    metrics = pd.DataFrame(models_metrics, columns=["Model", "Silhouette Score", "Davies-Bouldin Index", "Num Clusters"])

    # Normalizar métricas
    metrics["Silhouette Score Norm"] = (metrics["Silhouette Score"] - metrics["Silhouette Score"].min()) / (metrics["Silhouette Score"].max() - metrics["Silhouette Score"].min())
    metrics["Davies-Bouldin Index Norm"] = (metrics["Davies-Bouldin Index"].max() - metrics["Davies-Bouldin Index"]) / (metrics["Davies-Bouldin Index"].max() - metrics["Davies-Bouldin Index"].min())
    
    # Calcular puntuación combinada
    metrics["Combined Score"] = metrics[["Silhouette Score Norm", "Davies-Bouldin Index Norm"]].mean(axis=1)
    # Seleccionar el mejor modelo basado en el puntaje combinado
    best_model_info = metrics.loc[metrics["Combined Score"].idxmax()].to_dict()

    return metrics, best_model_info

In [58]:
# Función principal para ejecutar múltiples métodos de clustering
def automated_clustering(data, max_k=10, eps_range=(0.05, 0.2, 0.05), min_samples_range=(3, 12)):
    # Estandarizar los datos
    principal_df, optimal_components, explained_variance, cumulative_variance, pca_model = pca_pipeline(data, variance_threshold=0.95)

    X=principal_df
    # Lista para almacenar métricas de cada modelo
    models_metrics = []
    
    # Ejecutar K-means
    clusters_kmeans, silhouette_kmeans, davies_bouldin_kmeans, model_kmeans, ncluster_kmean = kmeans_clustering(X, max_k=max_k)
    models_metrics.append([model_kmeans, silhouette_kmeans, davies_bouldin_kmeans, ncluster_kmean])
    
    # Ejecutar DBSCAN
    clusters_dbscan, silhouette_dbscan, davies_bouldin_dbscan, model_dbscan, ncluster_dbscan = dbscan_clustering(X, eps_range=eps_range, min_samples_range=min_samples_range)
    models_metrics.append([model_dbscan, silhouette_dbscan, davies_bouldin_dbscan, ncluster_dbscan])
    
    # Ejecutar Clustering Jerárquico
    clusters_hierarchical, silhouette_hierarchical, davies_bouldin_hierarchical, model_hierarchical, ncluster_hierarchical = hierarchical_clustering(X)
    models_metrics.append([model_hierarchical, silhouette_hierarchical, davies_bouldin_hierarchical, ncluster_hierarchical])

    # Evaluar y seleccionar el mejor modelo
    metrics, best_model_info = evaluate_models(models_metrics)
    best_model_name = best_model_info["Model"]

    # Asignar las etiquetas de cluster al conjunto de datos según el mejor modelo
    if best_model_name == "K-Means":
        data["Cluster"] = clusters_kmeans
    elif best_model_name == "DBSCAN":
        data["Cluster"] = clusters_dbscan
    elif best_model_name == "Hierarchical":
        data["Cluster"] = clusters_hierarchical

    return metrics, best_model_info, data

## carga de datos 

In [59]:
file_path=data_dir("interim","estadisticas_edificios.csv")

In [60]:
data = pd.read_csv(file_path)
data.head()

,series_id,Cooling:Electricity [kW](Hourly)_mean,Cooling:Electricity [kW](Hourly)_std_dev,InteriorEquipment:Electricity [kW](Hourly)_mean,InteriorEquipment:Electricity [kW](Hourly)_std_dev
0,series_1,3.073887,6.226848,18.995908,7.265027
1,series_2,3.446951,6.596764,18.995908,7.265027
2,series_3,3.973240,7.146033,18.995908,7.265027
3,series_4,2.055950,4.688730,18.995908,7.265027
4,series_5,2.114192,4.797245,18.995908,7.265027


## PCA

In [53]:
X = standardize_data(data)
X.head()

,Cooling:Electricity [kW](Hourly)_mean,Cooling:Electricity [kW](Hourly)_std_dev,InteriorEquipment:Electricity [kW](Hourly)_mean,InteriorEquipment:Electricity [kW](Hourly)_std_dev
0,-0.486723,-0.555238,-0.468038,-0.560958
1,-0.483423,-0.550138,-0.468038,-0.560958
2,-0.478767,-0.542565,-0.468038,-0.560958
3,-0.495729,-0.576445,-0.468038,-0.560958
4,-0.495214,-0.574949,-0.468038,-0.560958


In [54]:
# Ejecutar el pipeline de PCA
principal_df, optimal_components, explained_variance, cumulative_variance, pca_model = pca_pipeline(data, variance_threshold=0.95)


In [57]:
principal_df

,PC1,PC2,PC3
0,-1.033624,0.044189,-0.066218
1,-1.029645,0.043682,-0.061656
2,-1.023853,0.042739,-0.054978
3,-1.047938,0.049979,-0.083314
4,-1.046985,0.049476,-0.082156
...,...,...,...
75,-1.237565,0.071661,0.059857
76,-1.235859,0.069978,0.062066
77,-1.234050,0.069329,0.064212
78,-1.245952,0.077263,0.049459


## CLUSTERING 

In [61]:
metrics, best_model_info, data=automated_clustering(data, max_k=10, eps_range=(0.05, 0.2, 0.05), min_samples_range=(3, 12))

C:\Users\ASUS TUF\.conda\envs\energy_consumption_architecture\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\ASUS TUF\.conda\envs\energy_consumption_architecture\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\ASUS TUF\.conda\envs\energy_consumption_architecture\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\ASUS TUF\.conda\

In [62]:
metrics

,Model,Silhouette Score,Davies-Bouldin Index,Num Clusters,Silhouette Score Norm,Davies-Bouldin Index Norm,Combined Score
0,K-Means,0.838543,0.216405,4,1.000000,1.000000,1.000000
1,DBSCAN,0.612861,0.540670,5,0.000000,0.056165,0.028082
2,Hierarchical,0.791535,0.559966,2,0.791709,0.000000,0.395855


In [63]:
best_model_info

{'Model': 'K-Means',
 'Silhouette Score': 0.838542519678743,
 'Davies-Bouldin Index': 0.21640469548252658,
 'Num Clusters': 4,
 'Silhouette Score Norm': 1.0,
 'Davies-Bouldin Index Norm': 1.0,
 'Combined Score': 1.0}

In [64]:
data[data["Cluster"]==1]

,series_id,Cooling:Electricity [kW](Hourly)_mean,Cooling:Electricity [kW](Hourly)_std_dev,InteriorEquipment:Electricity [kW](Hourly)_mean,InteriorEquipment:Electricity [kW](Hourly)_std_dev,Cluster
15,series_16,164.720807,188.343922,265.854193,116.539481,1
16,series_17,175.441358,194.702317,265.854193,116.539481,1
17,series_18,185.625160,199.371146,265.854193,116.539481,1
18,series_19,109.226257,181.623523,265.854193,116.539481,1
19,series_20,107.281889,177.627699,265.854193,116.539481,1


In [65]:
data.to_csv(data_dir("interim","clustering_all_characteristics.csv"),index=False)